In [9]:
import pandas as pd
#数据源:https://grouplens.org/datasets/movielens/；25M Movielens数据集中的ratings.csv,movies.csv,tags.csv
file_1 = pd.read_csv('./learning/L7/movielens/ml-latest/movies.csv')
file_2 = pd.read_csv('./learning/L7/movielens/ml-latest/ratings.csv')
file_3 = pd.read_csv('./learning/L7/movielens/ml-latest/tags.csv')
#合并3个文件生成DeepFM的数据集
file_3 = file_3.drop('timestamp' , axis =1) 
file_4 = pd.merge(file_2 , file_3)
file_5 = pd.merge(file_4 , file_1)
file_5.to_csv('./learning/L7/movielens/ml-latest/action2_movielens_2.csv' , index = 0)
print('Done')

Done


In [8]:
print(file_5.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 830492 entries, 0 to 830491
Data columns (total 7 columns):
userId       830492 non-null int64
movieId      830492 non-null int64
rating       830492 non-null float64
timestamp    830492 non-null int64
tag          830491 non-null object
title        830492 non-null object
genres       830492 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 50.7+ MB
None


In [25]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat,get_feature_names

#数据加载
data = pd.read_csv('./learning/L7/movielens/ml-latest/action2_movielens_2.csv' )
sparse_features = ['userId', 'movieId', 'timestamp', 'tag', 'title', 'genres']
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature].astype(str))
'''
'''
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)


[SparseFeat(name='userId', vocabulary_size=16358, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='userId', group_name='default_group'), SparseFeat(name='movieId', vocabulary_size=21942, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='movieId', group_name='default_group'), SparseFeat(name='timestamp', vocabulary_size=218612, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='timestamp', group_name='default_group'), SparseFeat(name='tag', vocabulary_size=57924, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='tag', group_name='default_group'), SparseFeat(name='title', vocabulary_size=21925, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='title', group_name='default_group'), SparseFeat(name='genres', vocabulary_size=1264, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='genres', group_name='default_group')]
Train on 531514 samples, validate on 132879 samples
Epoch 1/10


C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


531514/531514 [==============================] - 52s 98us/sample - loss: 0.6000 - mse: 0.5960 - val_loss: 0.1912 - val_mse: 0.1834
Epoch 2/10
531514/531514 [==============================] - 51s 96us/sample - loss: 0.0910 - mse: 0.0820 - val_loss: 0.1583 - val_mse: 0.1475
Epoch 3/10
531514/531514 [==============================] - 51s 96us/sample - loss: 0.0659 - mse: 0.0535 - val_loss: 0.1516 - val_mse: 0.1373
Epoch 4/10
531514/531514 [==============================] - 50s 94us/sample - loss: 0.0580 - mse: 0.0430 - val_loss: 0.1481 - val_mse: 0.1323
Epoch 5/10
531514/531514 [==============================] - 51s 96us/sample - loss: 0.0538 - mse: 0.0381 - val_loss: 0.1429 - val_mse: 0.1271
Epoch 6/10
531514/531514 [==============================] - 51s 96us/sample - loss: 0.0475 - mse: 0.0327 - val_loss: 0.1410 - val_mse: 0.1267
Epoch 7/10
531514/531514 [==============================] - 51s 96us/sample - loss: 0.0438 - mse: 0.0302 - val_loss: 0.1379 - val_mse: 0.1244
Epoch 8/10
531514